In [1]:
import pandas as pd

In [2]:
base_df = pd.read_csv('data/base.csv')

In [3]:
train_df = pd.read_csv('data/train.csv')

In [4]:
X_valid = pd.read_csv('data/validation.csv')

In [5]:
y_valid = pd.read_csv('data/validation_answer.csv')